# Student Advisory BOT – Middle East University

## Project Objective

The goal of this project is to develop a Generative AI-based Student Advisory BOT that provides personalized academic suggestions to students. By analyzing each student's academic history, strengths, and program details, the BOT recommends suitable elective courses and checks their availability for the upcoming semester. The BOT supports natural language queries from students to deliver context-aware guidance.

---

## Datasets Used

1. Student Enrollment Data  
   

2. Academic Program Data  
   

3. Class Schedule Data  
   

4. Term History Data  
   
---

## BOT Functionality

### 1. Understand the Student's Identity
- Extract the student name from a natural language query using NER (Named Entity Recognition).
- Retrieve the student profile from enrollment data including program and major.

### 2. Analyze Academic Strengths
- Evaluate completed courses and grade performance from the term history.
- Identify subjects and areas where the student performs well.

### 3. Suggest Relevant Electives
- Recommend 3 to 5 electives that align with the student's major and academic strengths.
- Avoid courses that the student has already completed.
- Match based on semantic similarity between elective descriptions and student strengths.

### 4. Check Upcoming Availability
- Confirm which of the recommended electives are available in the next semester.
- Filter suggestions based on class schedule data.

### 5. Deliver a Final Recommendation
- Provide a well-structured academic advisory response, including:
  - Student overview
  - Identified strengths
  - 3–5 relevant elective suggestions
  - Class schedule availability
  - Final recommendation summary

---

## Technology Stack

Language: Python  
Libraries and Tools:
- pandas, numpy – For data handling
- spacy, nltk – For NLP and name/entity recognition
- sentence-transformers – For semantic similarity between subjects and strengths
- openai or cohere – For generating natural language responses
- streamlit or gradio – For a simple web interface (optional)

---







## Importing Data and Modules

The following Python modules are imported for data analysis and manipulation:


In [1]:
import pandas as pd
import numpy as np

import os
os.chdir(r"C:\Users\USER\Desktop") # Lists all files in the current directory

# Load Excel files
enrollment_data = pd.read_excel("Student Enrollment.xlsx")
academic_program_data = pd.read_excel("Academic Program.xlsx")
class_schedule_data = pd.read_excel("Class Schedule.xlsx")
term_history_data = pd.read_excel("Term History.xlsx")

# Print shapes to understand the size difference
print(f"Enrollment Data: {enrollment_data.shape}")
print(f"Academic Program Data: {academic_program_data.shape}")
print(f"Class Schedule Data: {class_schedule_data.shape}")
print(f"Term History Data: {term_history_data.shape}")


Enrollment Data: (489, 16)
Academic Program Data: (31, 7)
Class Schedule Data: (2078, 26)
Term History Data: (347, 8)


## Exploring Dataset Structure

After importing the datasets, we inspect the column names and preview a few sample records from each dataset to understand their structure and contents.

In [2]:
# Display column names and sample records to understand structure
print("\n--- Enrollment Data Columns ---")
print(enrollment_data.columns)
print(enrollment_data.head(2))

print("\n--- Academic Program Data Columns ---")
print(academic_program_data.columns)
print(academic_program_data.head(2))

print("\n--- Class Schedule Data Columns ---")
print(class_schedule_data.columns)
print(class_schedule_data.head(2))

print("\n--- Term History Data Columns ---")
print(term_history_data.columns)
print(term_history_data.head(2))



--- Enrollment Data Columns ---
Index(['EMPLID', 'NAME_DISPLAY', 'ACAD_CAREER', 'INSTITUTION', 'STRM',
       'CLASS_NBR', 'UNT_TAKEN', 'ACAD_PROG', 'DESCR', 'CRSE_ID',
       'CRSE_GRADE_OFF', 'COURSE_TITLE_LONG', 'CUM_GPA', 'SUBJECT',
       'CATALOG_NBR', 'ACAD_ORG'],
      dtype='object')
    EMPLID          NAME_DISPLAY ACAD_CAREER INSTITUTION  STRM  CLASS_NBR  \
0  1075892  Yaman Ahmed Al saadi        UGRD         ADU  2301       3109   
1  1075892  Yaman Ahmed Al saadi        UGRD         ADU  2301       3128   

   UNT_TAKEN ACAD_PROG      DESCR  CRSE_ID CRSE_GRADE_OFF   COURSE_TITLE_LONG  \
0          3     ICOBA  ISL100(A)   666679             C+     Islamic Culture   
1          3     ICOBA     STT100   667089              D  General Statistics   

   CUM_GPA SUBJECT CATALOG_NBR ACAD_ORG  
0      2.5  ISL-UR  1000AD_M01   UC-CAS  
1      2.5  STT-UR  1000AD_M01   UC-COE  

--- Academic Program Data Columns ---
Index(['EMPLID', 'ACAD_CAREER', 'ACAD_PROG', 'PROG_STATUS', 'PRO

## Standardizing Column Names

To ensure consistent access and manipulation of data, we standardize column names across all datasets by converting them to lowercase, removing special characters, and replacing spaces with underscores.

A utility function is defined for this purpose:


In [3]:
# Function to clean column names (lowercase and replace spaces with underscores)
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-zA-Z0-9_]', '', regex=True)
    return df

# Clean each dataframe
enrollment_data = clean_columns(enrollment_data)
academic_program_data = clean_columns(academic_program_data)
class_schedule_data = clean_columns(class_schedule_data)
term_history_data = clean_columns(term_history_data)


## Verifying Cleaned Column Names

After standardizing the column names, we verify the new schema of each dataset by printing the list of columns. This helps ensure that all names are now consistent, lowercase, and free from special characters or spaces.


In [4]:
print("\n--- Enrollment Data Columns ---")
print(enrollment_data.columns.tolist())

print("\n--- Academic Program Data Columns ---")
print(academic_program_data.columns.tolist())

print("\n--- Class Schedule Data Columns ---")
print(class_schedule_data.columns.tolist())

print("\n--- Term History Data Columns ---")
print(term_history_data.columns.tolist())



--- Enrollment Data Columns ---
['emplid', 'name_display', 'acad_career', 'institution', 'strm', 'class_nbr', 'unt_taken', 'acad_prog', 'descr', 'crse_id', 'crse_grade_off', 'course_title_long', 'cum_gpa', 'subject', 'catalog_nbr', 'acad_org']

--- Academic Program Data Columns ---
['emplid', 'acad_career', 'acad_prog', 'prog_status', 'prog_action', 'admit_term', 'campus']

--- Class Schedule Data Columns ---
['course_id', 'term', 'offer_nbr', 'acad_group', 'subject', 'catalog', 'descr', 'class_nbr', 'cap_enrl', 'tot_enrl', 'acad_org', 'campus', 'section', 'id', 'role', 'facil_id', 'mtg_start', 'mtg_end', 'mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun', 'display_name']

--- Term History Data Columns ---
['emplid', 'tot_taken_prgrss', 'tot_passd_prgrss', 'acad_career', 'institution', 'acad_prog_primary', 'strm', 'term_gpa']


# Data Preprocessing

## Checking for Missing Values

Before performing any analysis, it's important to identify missing data within the datasets. We use the `isnull().sum()` function to count the number of missing values in each column of the datasets.

In [5]:
print("\n--- Missing Values: Enrollment Data ---")
print(enrollment_data.isnull().sum())

print("\n--- Missing Values: Academic Program Data ---")
print(academic_program_data.isnull().sum())

print("\n--- Missing Values: Class Schedule Data ---")
print(class_schedule_data.isnull().sum())

print("\n--- Missing Values: Term History Data ---")
print(term_history_data.isnull().sum())



--- Missing Values: Enrollment Data ---
emplid               0
name_display         0
acad_career          0
institution          0
strm                 0
class_nbr            0
unt_taken            0
acad_prog            0
descr                0
crse_id              0
crse_grade_off       0
course_title_long    0
cum_gpa              0
subject              0
catalog_nbr          0
acad_org             0
dtype: int64

--- Missing Values: Academic Program Data ---
emplid         0
acad_career    0
acad_prog      0
prog_status    0
prog_action    0
admit_term     0
campus         0
dtype: int64

--- Missing Values: Class Schedule Data ---
course_id         0
term              0
offer_nbr         0
acad_group        0
subject           0
catalog           0
descr             0
class_nbr         0
cap_enrl          0
tot_enrl          0
acad_org          0
campus            0
section           0
id                0
role              0
facil_id        130
mtg_start        83
mtg_end       

## Dropping Time-Related Columns from Class Schedule

The columns `mtg_start` and `mtg_end` in the `class_schedule_data` contain meeting time details, which are not required for our current analysis and recommendation logic.

We remove these columns to simplify the dataset and avoid unnecessary processing.

In [6]:
# Drop time-related columns
class_schedule_data.drop(['facil_id' ], axis=1, inplace=True)


In [7]:
print("\n--- Missing Values: Enrollment Data ---")
print(enrollment_data.isnull().sum())

print("\n--- Missing Values: Academic Program Data ---")
print(academic_program_data.isnull().sum())

print("\n--- Missing Values: Class Schedule Data ---")
print(class_schedule_data.isnull().sum())

print("\n--- Missing Values: Term History Data ---")
print(term_history_data.isnull().sum())



--- Missing Values: Enrollment Data ---
emplid               0
name_display         0
acad_career          0
institution          0
strm                 0
class_nbr            0
unt_taken            0
acad_prog            0
descr                0
crse_id              0
crse_grade_off       0
course_title_long    0
cum_gpa              0
subject              0
catalog_nbr          0
acad_org             0
dtype: int64

--- Missing Values: Academic Program Data ---
emplid         0
acad_career    0
acad_prog      0
prog_status    0
prog_action    0
admit_term     0
campus         0
dtype: int64

--- Missing Values: Class Schedule Data ---
course_id        0
term             0
offer_nbr        0
acad_group       0
subject          0
catalog          0
descr            0
class_nbr        0
cap_enrl         0
tot_enrl         0
acad_org         0
campus           0
section          0
id               0
role             0
mtg_start       83
mtg_end         83
mon              0
tues        

In [8]:
# Fill missing mtg_start and mtg_end in class_schedule_data with 'TBA'
class_schedule_data['mtg_start'].fillna('TBA', inplace=True)
class_schedule_data['mtg_end'].fillna('TBA', inplace=True)

print("\n--- Missing Values After Filling ---")
print(class_schedule_data[['mtg_start', 'mtg_end']].isnull().sum())




--- Missing Values After Filling ---
mtg_start    0
mtg_end      0
dtype: int64


In [9]:
print("\n--- Missing Values: Enrollment Data ---")
print(enrollment_data.isnull().sum())

print("\n--- Missing Values: Academic Program Data ---")
print(academic_program_data.isnull().sum())

print("\n--- Missing Values: Class Schedule Data ---")
print(class_schedule_data.isnull().sum())

print("\n--- Missing Values: Term History Data ---")
print(term_history_data.isnull().sum())



--- Missing Values: Enrollment Data ---
emplid               0
name_display         0
acad_career          0
institution          0
strm                 0
class_nbr            0
unt_taken            0
acad_prog            0
descr                0
crse_id              0
crse_grade_off       0
course_title_long    0
cum_gpa              0
subject              0
catalog_nbr          0
acad_org             0
dtype: int64

--- Missing Values: Academic Program Data ---
emplid         0
acad_career    0
acad_prog      0
prog_status    0
prog_action    0
admit_term     0
campus         0
dtype: int64

--- Missing Values: Class Schedule Data ---
course_id       0
term            0
offer_nbr       0
acad_group      0
subject         0
catalog         0
descr           0
class_nbr       0
cap_enrl        0
tot_enrl        0
acad_org        0
campus          0
section         0
id              0
role            0
mtg_start       0
mtg_end         0
mon             0
tues            0
wed         

In [10]:
print("\n--- Data Types: Enrollment Data ---")
print(enrollment_data.dtypes)

print("\n--- Data Types: Academic Program Data ---")
print(academic_program_data.dtypes)

print("\n--- Data Types: Class Schedule Data ---")
print(class_schedule_data.dtypes)

print("\n--- Data Types: Term History Data ---")
print(term_history_data.dtypes)



--- Data Types: Enrollment Data ---
emplid                 int64
name_display          object
acad_career           object
institution           object
strm                   int64
class_nbr              int64
unt_taken              int64
acad_prog             object
descr                 object
crse_id                int64
crse_grade_off        object
course_title_long     object
cum_gpa              float64
subject               object
catalog_nbr           object
acad_org              object
dtype: object

--- Data Types: Academic Program Data ---
emplid          int64
acad_career    object
acad_prog      object
prog_status    object
prog_action    object
admit_term      int64
campus         object
dtype: object

--- Data Types: Class Schedule Data ---
course_id        int64
term             int64
offer_nbr        int64
acad_group      object
subject         object
catalog         object
descr           object
class_nbr        int64
cap_enrl         int64
tot_enrl         int64
aca

## Merging Datasets for Unified Student Profile

To enable personalized academic recommendations, we merge multiple datasets into a single unified view containing enrollment details, academic programs, term-wise performance, and class schedule information.

### Step : Merge Enrollment Data with Academic Program Data

We begin by merging the student enrollment data with the academic program information using a left join on the columns `emplid`, `acad_career`, and `acad_prog`.


In [11]:
# Merge 1: Enrollment + Academic Program
merged_df = enrollment_data.merge(
    academic_program_data,
    on=["emplid", "acad_career", "acad_prog"],
    how="left"
)

# Merge 2: Add Term History
merged_df = merged_df.merge(
    term_history_data,
    on=["emplid", "acad_career", "strm"],
    how="left"
)

# Merge 3: Add Class Schedule
# Join on strm + class_nbr to match class within the correct term
merged_df = merged_df.merge(
    class_schedule_data,
    left_on=["strm", "class_nbr"],
    right_on=["term", "class_nbr"],
    how="left"
)

# Drop columns that are now redundant after merge (like 'term')
merged_df.drop(columns=["term"], inplace=True)

# Remove exact duplicates (if any)
merged_df = merged_df.drop_duplicates()

# Final overview
print("Final Merged Shape:", merged_df.shape)
print("Duplicate Rows Removed:", merged_df.duplicated().sum())


Final Merged Shape: (491, 48)
Duplicate Rows Removed: 0


In [12]:
merged_df.head()

,emplid,name_display,acad_career,institution_x,strm,class_nbr,unt_taken,acad_prog,descr_x,crse_id,...,mtg_start,mtg_end,mon,tues,wed,thurs,fri,sat,sun,display_name
0,1075892,Yaman Ahmed Al saadi,UGRD,ADU,2301,3109,3,ICOBA,ISL100(A),666679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1075892,Yaman Ahmed Al saadi,UGRD,ADU,2301,3128,3,ICOBA,STT100,667089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1075892,Yaman Ahmed Al saadi,UGRD,ADU,2301,3083,3,ICOBA,ITD100,667085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1075892,Yaman Ahmed Al saadi,UGRD,ADU,2301,1230,3,ICOBA,ENG200,667079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075892,Yaman Ahmed Al saadi,UGRD,ADU,2301,2245,0,ICOBA,MTG001(P),668592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_df.columns

Index(['emplid', 'name_display', 'acad_career', 'institution_x', 'strm',
       'class_nbr', 'unt_taken', 'acad_prog', 'descr_x', 'crse_id',
       'crse_grade_off', 'course_title_long', 'cum_gpa', 'subject_x',
       'catalog_nbr', 'acad_org_x', 'prog_status', 'prog_action', 'admit_term',
       'campus_x', 'tot_taken_prgrss', 'tot_passd_prgrss', 'institution_y',
       'acad_prog_primary', 'term_gpa', 'course_id', 'offer_nbr', 'acad_group',
       'subject_y', 'catalog', 'descr_y', 'cap_enrl', 'tot_enrl', 'acad_org_y',
       'campus_y', 'section', 'id', 'role', 'mtg_start', 'mtg_end', 'mon',
       'tues', 'wed', 'thurs', 'fri', 'sat', 'sun', 'display_name'],
      dtype='object')

In [14]:
merged_df.sample(7)

,emplid,name_display,acad_career,institution_x,strm,class_nbr,unt_taken,acad_prog,descr_x,crse_id,...,mtg_start,mtg_end,mon,tues,wed,thurs,fri,sat,sun,display_name
42,1079638,Afzaa Asif Mashkoora,UGRD,ADU,2402,1829,3,ICECS,SWE201,668432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
483,1095356,Nayef Abdulrahman Alameeri,PGRD,ADU,2301,3274,0,MEMA,ENG020(P),668705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,1085162,Ali Omar Alteneiji,UGRD,ADU,2401,2488,3,BSIT3,SWE201,668432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,1095175,Gavin Dilip Pawar,PGRD,ADU,2301,1339,3,PGDEB,EDC475(E),668311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,1093889,Marwa Mahmoud Salama,PGRD,ADU,2303,1414,3,PGDEB,EDC475(E),668311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,1082573,Mudhaffar Mabkhout Alameri,UGRD,ADU,2403,2550,3,ICOL,IILA337,668245,...,16:55:00,18:40:00,N,N,N,N,Y,N,N,Gaber Elshafey
214,1086695,Shahd Salah Ibrahim,UGRD,ADU,2301,2910,0,BAVI1,ENG002(P),668757,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
merged_df.describe()

,emplid,strm,class_nbr,unt_taken,crse_id,cum_gpa,admit_term,tot_taken_prgrss,tot_passd_prgrss,term_gpa,course_id,offer_nbr,cap_enrl,tot_enrl,section
count,4.910000e+02,491.000000,491.000000,491.000000,491.000000,491.000000,377.0,491.000000,491.000000,491.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,1.087238e+06,2339.818737,1965.588595,2.704684,667655.431772,3.104202,2301.0,25.892057,23.694501,2.478102,667701.361111,1.930556,23.375000,18.847222,36.666667
std,5.612059e+03,48.334820,665.599256,0.964872,749.963333,0.929832,0.0,18.960393,16.048407,1.491214,775.729594,1.154616,12.162558,12.427434,28.209478
min,1.069698e+06,2301.000000,1010.000000,0.000000,666674.000000,0.000000,2301.0,0.000000,0.000000,0.000000,666679.000000,1.000000,0.000000,1.000000,1.000000
25%,1.082662e+06,2301.000000,1336.500000,3.000000,666903.000000,2.750000,2301.0,10.000000,10.000000,1.625000,666855.750000,1.000000,18.000000,10.000000,22.000000
50%,1.087243e+06,2303.000000,1888.000000,3.000000,667648.000000,3.357000,2301.0,24.000000,24.000000,3.000000,668036.500000,1.000000,20.000000,17.000000,22.000000
75%,1.093362e+06,2401.000000,2557.000000,3.000000,668401.000000,3.814000,2301.0,38.000000,33.000000,3.750000,668448.250000,3.000000,30.000000,22.000000,51.250000
max,1.095446e+06,2403.000000,3354.000000,6.000000,668786.000000,4.000000,2301.0,78.000000,60.000000,4.000000,668772.000000,4.000000,70.000000,70.000000,99.000000


In [16]:
merged_df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
486    False
487    False
488    False
489    False
490    False
Length: 491, dtype: bool

## Checking Data Types of Columns

Before performing data transformations or analysis, it's important to verify the data types of each column. This ensures that operations like merging, filtering, or mathematical computations behave as expected.

We use the `dtypes` attribute to inspect column data types for each dataset:


In [17]:
merged_df.dtypes

emplid                 int64
name_display          object
acad_career           object
institution_x         object
strm                   int64
class_nbr              int64
unt_taken              int64
acad_prog             object
descr_x               object
crse_id                int64
crse_grade_off        object
course_title_long     object
cum_gpa              float64
subject_x             object
catalog_nbr           object
acad_org_x            object
prog_status           object
prog_action           object
admit_term           float64
campus_x              object
tot_taken_prgrss       int64
tot_passd_prgrss       int64
institution_y         object
acad_prog_primary     object
term_gpa             float64
course_id            float64
offer_nbr            float64
acad_group            object
subject_y             object
catalog               object
descr_y               object
cap_enrl             float64
tot_enrl             float64
acad_org_y            object
campus_y      

## Identifying Available Elective Courses

To provide accurate elective recommendations, we identify courses that are:


In [18]:
# Ensure the 'strm' column is numeric
merged_df["strm"] = pd.to_numeric(merged_df["strm"], errors="coerce")

# Drop rows with invalid/missing 'strm' values
merged_df = merged_df.dropna(subset=["strm"])

# Convert to integer (if needed)
merged_df["strm"] = merged_df["strm"].astype(int)

# Get the most recent term
current_term = merged_df["strm"].max()

# Filter for available courses (strm >= current term and capacity not yet full)
available_courses = merged_df[
    (merged_df["strm"] >= current_term) & 
    (merged_df["cap_enrl"] > merged_df["tot_enrl"])
]

# Get the number of available elective courses
available_course_count = available_courses["course_title_long"].dropna().nunique()

# Display the count of available elective courses
print(f"Total available elective courses: {available_course_count}")

# Optionally, display the titles of the available elective courses
elective_titles = available_courses["course_title_long"].dropna().unique()

print("Available elective courses:")
for title in elective_titles:
    print("-", title)


Total available elective courses: 38
Available elective courses:
- Principles of Managerial Accounting
- Principles of Macroeconomics
- Principles of Finance
- Operations Management
- Applied Management Science
- Database Management Systems
- Introduction to Digital Forensics
- Business Research Methods
- Consumer Protection Law
- Communication Skills in English II
- Primary Rights in Rem and Accessory Real Rights in Rem
- Introduction to Information and Digital Technology
- Labour Law and Social Securities Law
- Communication Theories
- Case Studies in PR and Advertising
- PR Media Production
- Introduction to Entrepreneurship
- Math for Life
- Bioinformatics
- Cancer Biology I
- Introduction to Aeronautics
- UAE and GCC Society
- Pre-Calculus
- General  Science
- Technical Communication for Work Place
- Genome Biology
- Principles of  Medical Genetics
- Accounting Information Systems
- Cost Accounting
- Artificial Intelligence for Engineers
- Cross-platform Mobile Application Develop

## Exploring Unique Universities and Course Types

To better understand the scope and diversity of the academic data, we analyze the number of unique universities and course types present in the merged dataset.

### Counting Unique Universities
We use the `institution_x` column to count how many different universities or institutions are represented.


In [19]:
# Count the number of unique universities (institution_x)
num_universities = merged_df["institution_x"].nunique()
print(f"Number of unique universities: {num_universities}")

# Count the number of unique course types (acad_prog)
num_course_types = merged_df["acad_prog"].nunique()
print(f"Number of unique course types: {num_course_types}")

# Optionally, display the unique values for each column to get a sense of them
unique_universities = merged_df["institution_x"].unique()
unique_course_types = merged_df["acad_prog"].unique()

print("\nUnique universities:")
print(unique_universities)

print("\nUnique course types:")
print(unique_course_types)


Number of unique universities: 1
Number of unique course types: 20

Unique universities:
['ADU']

Unique course types:
['ICOBA' 'ICECS' 'BHRM2' 'ICOL' 'DMC2' 'BMCA1' 'BSIT3' 'BBSL1' 'BAVI1'
 'BHGC1' 'BBAA2' 'BCMP2' 'LAWUA' 'M.CBA' 'MSCME' 'MPMA' 'PGDEB' 'LAWVA'
 'EDLDB' 'MEMA']


## Subject Mapping by Academic Program

To enable personalized academic advisory, it's essential to understand the subject offerings under each academic program. The following logic groups subjects (courses) by their respective `acad_prog` (academic program codes).

### Step : Define List of Course Types
We create a predefined list of academic programs for which we want to extract subject information.


In [20]:
# List of course types
course_types = ['ICOBA', 'ICECS', 'BHRM2', 'ICOL', 'DMC2', 'BMCA1', 'BSIT3', 
                'BBSL1', 'BAVI1', 'BHGC1', 'BBAA2', 'BCMP2', 'LAWUA', 'M.CBA', 
                'MSCME', 'MPMA', 'PGDEB', 'LAWVA', 'EDLDB', 'MEMA']

# Loop through each course type and display the subjects as a comma-separated list
for course_type in course_types:
    # Filter the data for the current course type
    subjects = merged_df[merged_df["acad_prog"] == course_type]["descr_x"].unique()
    
    # Join subjects into a single paragraph separated by commas
    subjects_paragraph = ", ".join(subjects)
    
    # Print the course type and its subjects in a paragraph format
    print(f"Subjects under course type {course_type}:")
    print(f"{subjects_paragraph}\n")
    print("="*50 + "\n")  # Separator between course types


Subjects under course type ICOBA:
ISL100(A), STT100, ITD100, ENG200, MTG001(P), FWS212, MTG100, FWS205, BUS102, FWS100, MTB101, ACC200, BUS204, ECO201, MIS200, MGT255, MKT200, ACC201, ECO202, FIN200, MGT308, BUS306


Subjects under course type ICECS:
USS001(P), MTG002(P), ECT200, PHY009(P), STT100, ENG200, GES201, CSE210, ISL100(E), ARL101-E, FWS205, MTH100(R), MTT202, PBH101, STT201, MTT101, COE102, SWE201, CSC202, CSC302, CSC307, CSE300, ISL100(A), ARL101(A), ENG102(R), FWS100, ECS200, PBH110, MTT102, CHE205


Subjects under course type BHRM2:
MTG100, ISL100(A), STT100, ENG002(P)


Subjects under course type ICOL:
MTGL001(P), ISL100(A), ENGL001(P), INLA105, PGLA225, PCLA110, ARL101(A), PELA219, ENG100(AA), COLA200, ADLA205, INLA210, PPLA329, RCLA310, BFLA348, SWLA440, ITD100(AA), CCLA320, PPLA450, JILA344, SULA209, ECLA301, HPLA150, FWS100(AA), BBLA431, CPLA288, ENG200(AA), IILA337, ORLA477, LSLA335, IRLA280, IFLA218


Subjects under course type DMC2:
ISL100(A), ENG002(P), MTG001(P),

## Count of Subjects by Academic Program

To support better academic advisory and curriculum overview, this section of the script counts the number of unique subjects (courses) offered under each academic program 'acad_prog'.

### Defining Course Types
We define a list of academic program codes to analyze.

In [21]:
# List of course types
course_types = ['ICOBA', 'ICECS', 'BHRM2', 'ICOL', 'DMC2', 'BMCA1', 'BSIT3', 
                'BBSL1', 'BAVI1', 'BHGC1', 'BBAA2', 'BCMP2', 'LAWUA', 'M.CBA', 
                'MSCME', 'MPMA', 'PGDEB', 'LAWVA', 'EDLDB', 'MEMA']

# Loop through each course type and count the number of subjects
for course_type in course_types:
    # Filter the data for the current course type
    subjects = merged_df[merged_df["acad_prog"] == course_type]["descr_x"].unique()
    
    # Count the number of subjects
    num_subjects = len(subjects)
    
    # Print the course type and the number of subjects
    print(f"Course type: {course_type}")
    print(f"Number of subjects: {num_subjects}\n")
    print("="*50 + "\n")  # Separator between course types


Course type: ICOBA
Number of subjects: 22


Course type: ICECS
Number of subjects: 30


Course type: BHRM2
Number of subjects: 4


Course type: ICOL
Number of subjects: 32


Course type: DMC2
Number of subjects: 17


Course type: BMCA1
Number of subjects: 26


Course type: BSIT3
Number of subjects: 16


Course type: BBSL1
Number of subjects: 19


Course type: BAVI1
Number of subjects: 15


Course type: BHGC1
Number of subjects: 21


Course type: BBAA2
Number of subjects: 22


Course type: BCMP2
Number of subjects: 26


Course type: LAWUA
Number of subjects: 12


Course type: M.CBA
Number of subjects: 23


Course type: MSCME
Number of subjects: 1


Course type: MPMA
Number of subjects: 10


Course type: PGDEB
Number of subjects: 9


Course type: LAWVA
Number of subjects: 10


Course type: EDLDB
Number of subjects: 1


Course type: MEMA
Number of subjects: 2




## Subject List by Full Course Name

This section provides a detailed breakdown of subjects grouped by their respective academic programs. Each program is identified using its course code and full name, followed by a comma-separated list of unique subjects offered within that program.

---

### Step : Course Code to Full Name Mapping
We define a dictionary that maps abbreviated course codes to their full course names for better readability and user understanding.


In [22]:
# Dictionary for full course names
course_fullnames = {
    'ICOBA': 'International & Commercial Business Administration',
    'ICECS': 'International & Computer Engineering Science',
    'BHRM2': 'Human Resource Management',
    'ICOL': 'International Communication & Language',
    'DMC2': 'Digital Media Communications',
    'BMCA1': 'Business Management and Computer Applications',
    'BSIT3': 'Bachelor of Science in Information Technology',
    'BBSL1': 'Bachelor of Business Studies and Law',
    'BAVI1': 'Bachelor of Aviation',
    'BHGC1': 'Bachelor of Hospitality & General Management',
    'BBAA2': 'Bachelor of Business Administration',
    'BCMP2': 'Bachelor of Computer Programming',
    'LAWUA': 'Law Undergraduate Program',
    'M.CBA': 'Master of Computer Business Administration',
    'MSCME': 'Master of Science in Computer and Engineering',
    'MPMA': 'Master of Public Administration',
    'PGDEB': 'Post Graduate Diploma in Education and Business',
    'LAWVA': 'Law Postgraduate Program',
    'EDLDB': 'Educational Leadership and Development Program',
    'MEMA': 'Master of Engineering and Management Administration'
}

for i in course_fullnames:
    print(course_fullnames[i])
# List of course types
course_types = ['ICOBA', 'ICECS', 'BHRM2', 'ICOL', 'DMC2', 'BMCA1', 'BSIT3', 
                'BBSL1', 'BAVI1', 'BHGC1', 'BBAA2', 'BCMP2', 'LAWUA', 'M.CBA', 
                'MSCME', 'MPMA', 'PGDEB', 'LAWVA', 'EDLDB', 'MEMA']

# Loop through each course type and display the subjects
for course_type in course_types:
    # Fetch the course full name
    full_course_name = course_fullnames.get(course_type, "Course name not available")
    
    # Filter the data for the current course type
    subjects = merged_df[merged_df["acad_prog"] == course_type]["descr_x"].unique()
    
    # Join subjects into a single string separated by commas
    subjects_paragraph = ", ".join(subjects)
    
    # Print the course full name and subjects in the desired format
    print(f"Subjects under course type {full_course_name}:")
    print(f"{subjects_paragraph}\n")
    print("="*50 + "\n")  # Separator between course types


International & Commercial Business Administration
International & Computer Engineering Science
Human Resource Management
International Communication & Language
Digital Media Communications
Business Management and Computer Applications
Bachelor of Science in Information Technology
Bachelor of Business Studies and Law
Bachelor of Aviation
Bachelor of Hospitality & General Management
Bachelor of Business Administration
Bachelor of Computer Programming
Law Undergraduate Program
Master of Computer Business Administration
Master of Science in Computer and Engineering
Master of Public Administration
Post Graduate Diploma in Education and Business
Law Postgraduate Program
Educational Leadership and Development Program
Master of Engineering and Management Administration
Subjects under course type International & Commercial Business Administration:
ISL100(A), STT100, ITD100, ENG200, MTG001(P), FWS212, MTG100, FWS205, BUS102, FWS100, MTB101, ACC200, BUS204, ECO201, MIS200, MGT255, MKT200, ACC201

# Extract merged_df file to Desktop

In [25]:
merged_df.to_excel(r"C:\Users\USER\Desktop\merged_df.xlsx", index=False)


In [26]:
import os
file_path = r"C:\Users\USER\Desktop\merged_df.xlsx"
if os.path.exists(file_path):
    print("File saved successfully!")
else:
    print("Failed to save the file.")


File saved successfully!


# 🎓 Elective Recommendation System using NLP

This project recommends the most relevant **elective subjects** to students based on their academic background using **TF-IDF** and **cosine similarity**. The system analyzes the subjects a student has already completed and finds the most semantically similar electives from a predefined list.


In [76]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  Replace this with your full electives dataset including modules
electives_df = pd.DataFrame({
    'course_title_long': [
        "Principles of Managerial Accounting", "Principles of Macroeconomics", "Principles of Finance",
        "Operations Management", "Applied Management Science", "Database Management Systems",
        "Introduction to Digital Forensics", "Business Research Methods", "Consumer Protection Law",
        "Communication Skills in English II", "Primary Rights in Rem and Accessory Real Rights in Rem",
        "Introduction to Information and Digital Technology", "Labour Law and Social Securities Law",
        "Communication Theories", "Case Studies in PR and Advertising", "PR Media Production",
        "Introduction to Entrepreneurship", "Math for Life", "Bioinformatics", "Cancer Biology I",
        "Introduction to Aeronautics", "UAE and GCC Society", "Pre-Calculus", "General Science",
        "Technical Communication for Work Place", "Genome Biology", "Principles of Medical Genetics",
        "Accounting Information Systems", "Cost Accounting", "Artificial Intelligence for Engineers",
        "Cross-platform Mobile Application Develop.", "Calculus I", "Probability and Stochastic Processes",
        "Project Scheduling and Time Management", "Project Costing and Financial Management",
        "Leadership and Communication", "Methods of Teaching Math", "Business Ethics and Corporate Governance"
    ],
    'module': [
        "Accounting", "Economics", "Finance", "Operations", "Management", "IT & Systems",
        "Cybersecurity", "Research", "Law", "Communication", "Law", "IT & Systems", "Law",
        "Communication", "Advertising", "Media", "Entrepreneurship", "Mathematics", "Biotech",
        "Biotech", "Engineering", "Sociology", "Mathematics", "General Science", "Communication",
        "Biotech", "Biotech", "Accounting", "Accounting", "Engineering", "App Development",
        "Mathematics", "Mathematics", "Project Management", "Finance", "Leadership", "Education", "Business Ethics"
    ]
})

#  Main function
def recommend_electives(student_name, merged_df, electives_df):
    student_data = merged_df[merged_df['name_display'].str.strip().str.lower() == student_name.strip().lower()]
    
    if student_data.empty:
        print(" Student not found in the records.")
        return

    std_id = student_data['emplid'].iloc[0]
    academic_program = student_data['acad_prog'].iloc[0]
    gpa = student_data['cum_gpa'].iloc[0] if 'cum_gpa' in student_data.columns else 'N/A'
    
    print(f"\n Name: {student_name}")
    print(f" Student ID: {std_id}")
    print(f" Academic Program: {academic_program}")
    print(f" Cumulative GPA: {gpa}")
    
    subjects_taken = student_data[['course_title_long', 'subject_x', 'crse_grade_off']].drop_duplicates().reset_index(drop=True)
    print("\n Subjects Taken:")
    print(subjects_taken.to_string(index=False))

    taken_courses_set = set(subjects_taken['course_title_long'].str.strip().str.lower())
    eligible_electives = electives_df[~electives_df['course_title_long'].str.strip().str.lower().isin(taken_courses_set)].copy()

    #  Enhanced NLP similarity match using TF-IDF (considering domain relevance)
    taken_subjects_text = ' '.join(subjects_taken['course_title_long'].dropna().tolist())
    elective_texts = eligible_electives['course_title_long'].tolist()
    
    # Combine subject list with elective titles for TF-IDF transformation
    corpus = [taken_subjects_text] + elective_texts
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
    eligible_electives['similarity_score'] = similarity_scores
    eligible_electives['similarity_score'] = eligible_electives['similarity_score'].round(2)
    
    # Filter electives based on domain relevance (e.g., LAWVA)
    if "LAW" in academic_program.upper():
        eligible_electives = eligible_electives[eligible_electives['module'].str.contains("Law", case=False)]

    # Sorting electives by similarity score
    top_electives = eligible_electives.sort_values(by='similarity_score', ascending=False).head(5)

    print("\n Recommended Elective Subjects (with Day, Time & Module):")
    for _, row in top_electives.iterrows():
        title = row['course_title_long']
        module = row['module']
        class_schedule = merged_df[merged_df['course_title_long'].str.strip().str.lower() == title.strip().lower()]
        
        if not class_schedule.empty:
            sched_row = class_schedule.iloc[0]
            days = [day_col.capitalize() for day_col in ['mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
                    if str(sched_row[day_col]).strip().upper() == 'Y']
            time_info = f"{sched_row['mtg_start']} - {sched_row['mtg_end']}" if pd.notna(sched_row['mtg_start']) else "To Be Announced"
            day_str = ', '.join(days) if days else "To Be Announced"
        else:
            time_info = "To Be Announced"
            day_str = "To Be Announced"

        print(f" - {title} ({module})\n   🕒 Time: {time_info}\n    Days: {day_str}\n")

    print("These electives were suggested based on your academic history and NLP-based semantic similarity.\n")

# Example usage (you must load `merged_df` beforehand)
print("!Hello \n I am your Academic Advisory BOT")
print("I will get recomended elective subjects recommendations based on your academic history.\n")

student_name = input("Please enter your full name: ")
recommend_electives(student_name, merged_df, electives_df)




!Hello 
 I am your Academic Advisory BOT
I will get recomended elective subjects recommendations based on your academic history.



Please enter your full name:  Humaid Khalifa Alshehhi



 Name: Humaid Khalifa Alshehhi
 Student ID: 1093979
 Academic Program: LAWVA
 Cumulative GPA: 3.75

 Subjects Taken:
            course_title_long subject_x crse_grade_off
    International Private Law       LAW             B+
International Trade Contracts       LAW              A
               Commercial Law       LAW             B+
          Civil Procedure Law       LAW             B+
Jurisprudence of Transactions       LAW              A
       Civil Transactions Law       LAW              A
      Consumer Protection Law       LAW              A
               Legal Research       LAW              A
                Thesis Part-A       LAW              P
                Thesis Part-B       LAW              P

 Recommended Elective Subjects (with Day, Time & Module):
 - Labour Law and Social Securities Law (Law)
   🕒 Time: 18:50:00 - 20:35:00
    Days: Tues, Thurs

 - Primary Rights in Rem and Accessory Real Rights in Rem (Law)
   🕒 Time: 16:55:00 - 18:40:00
    Days: Mon, Wed

The

## Conclusion

The development of the Generative AI-based Student Advisory BOT successfully demonstrates the potential of intelligent systems to support students with personalized academic guidance. By integrating natural language processing with structured student, program, and scheduling data, the BOT can provide contextual, relevant, and actionable recommendations in real-time.

The system is designed to perform the following key functions:

- Identify and validate student identity using Enrollment and Term History data.
- Analyze academic strengths and interests through performance trends and completed coursework.
- Recommend electives that align with the student's academic background and interests.
- Verify the availability of recommended electives in the upcoming semester based on the Class Schedule dataset.
- Generate coherent, conversational responses that enhance the student advisory experience.

This project showcases the ability to synthesize multi-source datasets and apply NLP techniques to understand natural queries and return personalized, insightful academic advice. The BOT's architecture is scalable, modular, and ready for integration with web-based platforms or university portals.

The Student Advisory BOT not only reduces the workload on academic counselors but also empowers students to make informed decisions about their academic journey. With further improvements—such as incorporating deep learning-based embeddings, student feedback loops, and multilingual capabilities—the system can become a comprehensive advisory companion for universities across the region.

This solution reflects the future of student engagement in higher education, where intelligent systems augment human advisors, ensuring timely, consistent, and data-driven guidance.
